# 기획자나 마케터가 되어 이벤트 댓글을 분석해 보자!

## 내가 만약 수백 수천개의 댓글을 다 읽어봐야 한다면?

## 댓글 속에 제품에 대한 관심을 빈도 수로 추출해야 한다면?
  
  * 쇼핑몰에서 제품 관련 이벤트 진행시 어떤 제품을 고객이 선호하는지 알고 싶다면?
  * 고객 DB와 연계할 수 있다면 이벤트 혹은 마케팅 세그먼트로 활용해 볼 수도 있지 않을까?
  
## 향후 마케팅이나 전략을 어떻게 세워야 할까?

In [1]:
# 라이브러리 로드

import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

%matplotlib inline
# 시각화 결과가 선명하게 표시되도록
%config InlineBackend.figure_format = 'retina'

In [2]:
# 시각화를 위한 한글폰트 설정

# Window 한글폰트 설정
plt.rc("font", family="Malgun Gothic")

In [3]:
# 크롤링한 파일 로드하기

df = pd.read_csv("inflearn-event.csv")
df.shape

(100, 1)

In [4]:
df.head()

,text
0,올해 이직하기!
1,스프링과 리액트 기초부터 인프런을 통해 공부하기!!
2,인터랙티브웹 강의 수강
3,타입스크립트 정복
4,2021 인프런 새해 다짐 이벤트가 종료되었습니다.참여해주신 모든 분들 감사합니다!...


In [5]:
df.tail()

,text
95,쿠버네티스자바 스프링파이썬 코딩모두 이뤄내고 취업까지 성공해서 멋지게 다시 돌아오겠...
96,"[초급~활용] ""Oh!"" 감탄사가 나오는 역동적인 인터랙티브 웹사이트를 만들어요.-..."
97,1. Spring Boot Security 마스터학2. JPA 마스터하기!!3. 이...
98,우선 지금 듣고있는 자료구조 알고리즘 강의를 끝내고 자바 spring 강의들도 들어...
99,공공데이터로 파이썬 데이터분석 시작하기올해는 꼭 듣고싶어요!!!2021년 한 해는 ...


# 데이터 전처리

### 네트워크 오류 등으로 발생한 중복 입력 값을 제거
  * 빈도 수 중복을 방지하기 위해

In [6]:
print(df.shape)
df = df.drop_duplicates(["text"], keep="last")
print(df.shape)

(100, 1)
(100, 1)


### 원본은 따로 보존

In [8]:
# 전처리 전에 원본을 보존하기 위해 복사해 둠
df["origin_text"] = df["text"]
df.head

<bound method NDFrame.head of                                                  text  \
0                                            올해 이직하기!   
1                        스프링과 리액트 기초부터 인프런을 통해 공부하기!!   
2                                        인터랙티브웹 강의 수강   
3                                           타입스크립트 정복   
4   2021 인프런 새해 다짐 이벤트가 종료되었습니다.참여해주신 모든 분들 감사합니다!...   
..                                                ...   
95  쿠버네티스자바 스프링파이썬 코딩모두 이뤄내고 취업까지 성공해서 멋지게 다시 돌아오겠...   
96  [초급~활용] "Oh!" 감탄사가 나오는 역동적인 인터랙티브 웹사이트를 만들어요.-...   
97  1. Spring Boot Security 마스터학2. JPA 마스터하기!!3. 이...   
98  우선 지금 듣고있는 자료구조 알고리즘 강의를 끝내고 자바 spring 강의들도 들어...   
99  공공데이터로 파이썬 데이터분석 시작하기올해는 꼭 듣고싶어요!!!2021년 한 해는 ...   

                                          origin_text  
0                                            올해 이직하기!  
1                        스프링과 리액트 기초부터 인프런을 통해 공부하기!!  
2                                        인터랙티브웹 강의 수강  
3                                           타입스크립트 정복  
4   2

### 소문자 변환

In [9]:
# 파이썬은 대소문자를 구분하기 때문에 데이터 필터링을 위해 대문자를 모두 소문자로 변경
df["text"] = df["text"].str.lower()

In [10]:
# 같은 의미의 단어를 하나로 통일 예) python => 파이썬
df["text"] = df["text"].str.replace(
    "python", "파이썬").str.replace(
    "pandas", "판다스").str.replace(
    "javascript", "자바스크립트").str.replace(
    "java", "자바").str.replace(
    "react", "리액트")

### 텍스트로 관심 강의 분리

  * 혹시 이벤트 조건에 해시테그로 "관심강의" 다는것에 대하여

In [16]:
df["course"] = df["text"].apply(lambda x: x.split("관심강의")[-1])
df["course"] = df["course"].apply(lambda x: x.split("관심 강의")[-1])
df["course"] = df["course"].apply(lambda x: x.split("관심 강좌")[-1])
df["course"] = df["course"].str.replace(":","")
df["course"].head()

0                                             올해 이직하기!
1                         스프링과 리액트 기초부터 인프런을 통해 공부하기!!
2                                         인터랙티브웹 강의 수강
3                                            타입스크립트 정복
4    2021 인프런 새해 다짐 이벤트가 종료되었습니다.참여해주신 모든 분들 감사합니다!...
Name: course, dtype: object

In [17]:
df["course"].tail()

95    쿠버네티스자바 스프링파이썬 코딩모두 이뤄내고 취업까지 성공해서 멋지게 다시 돌아오겠...
96    [초급~활용] "oh!" 감탄사가 나오는 역동적인 인터랙티브 웹사이트를 만들어요.-...
97    1. spring boot security 마스터학2. jpa 마스터하기!!3. 이...
98    우선 지금 듣고있는 자료구조 알고리즘 강의를 끝내고 자바 spring 강의들도 들어...
99    공공데이터로 파이썬 데이터분석 시작하기올해는 꼭 듣고싶어요!!!2021년 한 해는 ...
Name: course, dtype: object

### 띄어 쓰기를 제거한 텍스트에서 키워드 추출

In [34]:
search_keyword = ['머신러닝', '딥러닝', '파이썬', '판다스', '공공데이터', 
                 'django', '크롤링', '시각화' '데이터분석', '웹개발', 
                 '엑셀', 'c', '자바', '자바스크립트', 'node', 'vue', '리엑트', '데이터분석',
                 '리액트']

for keyword in search_keyword:
    df[keyword] = df["course"].str.contains(keyword)

In [35]:
df.head()

,text,origin_text,course,머신러닝,딥러닝,파이썬,판다스,공공데이터,django,크롤링,...,웹개발,엑셀,c,자바,자바스크립트,node,vue,리엑트,데이터분석,리액트
0,올해 이직하기!,올해 이직하기!,올해 이직하기!,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,스프링과 리액트 기초부터 인프런을 통해 공부하기!!,스프링과 리액트 기초부터 인프런을 통해 공부하기!!,스프링과 리액트 기초부터 인프런을 통해 공부하기!!,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,인터랙티브웹 강의 수강,인터랙티브웹 강의 수강,인터랙티브웹 강의 수강,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,타입스크립트 정복,타입스크립트 정복,타입스크립트 정복,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,2021 인프런 새해 다짐 이벤트가 종료되었습니다.참여해주신 모든 분들 감사합니다!...,2021 인프런 새해 다짐 이벤트가 종료되었습니다.참여해주신 모든 분들 감사합니다!...,2021 인프런 새해 다짐 이벤트가 종료되었습니다.참여해주신 모든 분들 감사합니다!...,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [36]:
df_python = df[df["text"].str.contains("파이썬|공공데이터|판다스|데이터분석")].copy()
df_python.head()

,text,origin_text,course,머신러닝,딥러닝,파이썬,판다스,공공데이터,django,크롤링,...,웹개발,엑셀,c,자바,자바스크립트,node,vue,리엑트,데이터분석,리액트
7,‘올해는 꼭 배우고 싶은 인프런 강의'vue.js 시작하기 - age of vue....,‘올해는 꼭 배우고 싶은 인프런 강의'Vue.js 시작하기 - Age of Vue....,‘올해는 꼭 배우고 싶은 인프런 강의'vue.js 시작하기 - age of vue....,True,False,True,False,True,False,False,...,True,False,True,True,True,False,True,False,False,True
8,공공데이터로 파이썬 데이터 분석하기강의를 꼭 듣고 싶다 / 한 분야에전문가되기,공공데이터로 파이썬 데이터 분석하기강의를 꼭 듣고 싶다 / 한 분야에전문가되기,공공데이터로 파이썬 데이터 분석하기강의를 꼭 듣고 싶다 / 한 분야에전문가되기,False,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
11,올해는 꼭 듣고 싶은 인프런 강의-파이썬 플라스크(flask)로 웹 서비스 만들기-...,올해는 꼭 듣고 싶은 인프런 강의-파이썬 플라스크(Flask)로 웹 서비스 만들기-...,올해는 꼭 듣고 싶은 인프런 강의-파이썬 플라스크(flask)로 웹 서비스 만들기-...,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
22,올해는 작년에 포기했던 파이썬을 다시 공부해서 직장에서 더 나은 업무스킬을 만들어갈...,올해는 작년에 포기했던 파이썬을 다시 공부해서 직장에서 더 나은 업무스킬을 만들어갈...,올해는 작년에 포기했던 파이썬을 다시 공부해서 직장에서 더 나은 업무스킬을 만들어갈...,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
27,올해는 판다스 데이터 분석과 머신러닝을 공부해보고 싶어요..시간이 허락한다면 빅데이...,올해는 판다스 데이터 분석과 머신러닝을 공부해보고 싶어요..시간이 허락한다면 빅데이...,올해는 판다스 데이터 분석과 머신러닝을 공부해보고 싶어요..시간이 허락한다면 빅데이...,True,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [37]:
df_python.shape

(14, 21)

In [39]:
df.head()

,text,origin_text,course,머신러닝,딥러닝,파이썬,판다스,공공데이터,django,크롤링,...,웹개발,엑셀,c,자바,자바스크립트,node,vue,리엑트,데이터분석,리액트
0,올해 이직하기!,올해 이직하기!,올해 이직하기!,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,스프링과 리액트 기초부터 인프런을 통해 공부하기!!,스프링과 리액트 기초부터 인프런을 통해 공부하기!!,스프링과 리액트 기초부터 인프런을 통해 공부하기!!,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,인터랙티브웹 강의 수강,인터랙티브웹 강의 수강,인터랙티브웹 강의 수강,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,타입스크립트 정복,타입스크립트 정복,타입스크립트 정복,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,2021 인프런 새해 다짐 이벤트가 종료되었습니다.참여해주신 모든 분들 감사합니다!...,2021 인프런 새해 다짐 이벤트가 종료되었습니다.참여해주신 모든 분들 감사합니다!...,2021 인프런 새해 다짐 이벤트가 종료되었습니다.참여해주신 모든 분들 감사합니다!...,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [40]:
df[search_keyword]

,머신러닝,딥러닝,파이썬,판다스,공공데이터,django,크롤링,시각화데이터분석,웹개발,엑셀,c,자바,자바스크립트,node,vue,리엑트,데이터분석,리액트
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False
96,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
97,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
98,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False


In [41]:
df[search_keyword].sum() #True 값의 빈도수

머신러닝         4
딥러닝          2
파이썬         13
판다스          1
공공데이터        5
django       1
크롤링          1
시각화데이터분석     0
웹개발          2
엑셀           1
c           27
자바          15
자바스크립트       9
node         4
vue          4
리엑트          0
데이터분석        2
리액트         13
dtype: int64

In [42]:
df[search_keyword].sum().sort_values(ascending=False)

c           27
자바          15
리액트         13
파이썬         13
자바스크립트       9
공공데이터        5
머신러닝         4
node         4
vue          4
데이터분석        2
딥러닝          2
웹개발          2
엑셀           1
크롤링          1
django       1
판다스          1
시각화데이터분석     0
리엑트          0
dtype: int64

In [44]:
# 파이썬 들어간 글들만 보고싶다.

text = df.loc[(df["파이썬"] == True), "text"]
for t in text:
    print("-"*20)
    print(t)

--------------------
‘올해는 꼭 배우고 싶은 인프런 강의'vue.js 시작하기 - age of vue.js실전 리액트 프로그래밍가장 빠른 풀스택: 파이썬 백엔드와 웹기술 기본웹 게임을 만들며 배우는 자바스크립트인터랙티브 웹 개발 제대로 시작하기html+css+jquery 포트폴리오 실전 퍼블리싱공공데이터로 파이썬 데이터 분석 시작하기파이썬 머신러닝 완벽 가이드레트로의 유니티 c# 게임 프로그래밍 에센스유니티 강좌 [쯔꾸르풍 게임 제작하기][c#과 유니티로 만드는 mmorpg 게임 개발 시리즈]'2021년 공부 목표'풀스택 웹개발파이썬 머신러닝유니티 게임개발
--------------------
공공데이터로 파이썬 데이터 분석하기강의를 꼭 듣고 싶다 / 한 분야에전문가되기
--------------------
올해는 꼭 듣고 싶은 인프런 강의-파이썬 플라스크(flask)로 웹 서비스 만들기-파이썬을 활용한 데이터분석과 it보안- 파이썬 알고리즘 문제풀이 (코딩테스트 대비)-구글 알고리즘 문제 풀이2021년 공부 목표- 파이썬 flask와 sqlite를 활용해서 웹애플리케이션 만들기- 자료구조와 알고리즘 공부 병행- 애플리케이션 개발- 부트캠프 or 웹 개발자로서 커리어 시작하기💪
--------------------
올해는 작년에 포기했던 파이썬을 다시 공부해서 직장에서 더 나은 업무스킬을 만들어갈 예정입니다!
--------------------
듣고싶은것 : 공공데이터로 파이썬 데이터 분석 시작하기파이썬을 제대로 배워보는 한해가 되었으면 합니다
--------------------
올해는 꼭 배우고 싶은 인프런 강의: 파이썬 활용 프로젝트 - 웹 크롤링, 웹 개발올해는파이썬을 본격적으로 활용해보는 해로 만드는 것이 목표입니다.작을지라도 완성된 프로젝트 수를 늘리려 합니다~^^;
--------------------
올해는 꼭 배우고 싶은 인프런 강의 : 파이썬을 이용한 공공데이터활용2021년 공부 목표 : 하루 10분씩이라도 강의 필참  화

In [45]:
# 데이터분석 들어간 글들만 보고싶다.

text = df.loc[(df["데이터분석"] == True), "text"]
for t in text:
    print("-"*20)
    print(t)

--------------------
올해는 꼭 듣고 싶은 인프런 강의-파이썬 플라스크(flask)로 웹 서비스 만들기-파이썬을 활용한 데이터분석과 it보안- 파이썬 알고리즘 문제풀이 (코딩테스트 대비)-구글 알고리즘 문제 풀이2021년 공부 목표- 파이썬 flask와 sqlite를 활용해서 웹애플리케이션 만들기- 자료구조와 알고리즘 공부 병행- 애플리케이션 개발- 부트캠프 or 웹 개발자로서 커리어 시작하기💪
--------------------
공공데이터로 파이썬 데이터분석 시작하기올해는 꼭 듣고싶어요!!!2021년 한 해는 도전하고, 꾸준히 노력하는 한 해가 되는게 목표입니다!!
